In [0]:
!pip install -U -q PyDrive

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1kO1y5CRbabnobaZ4RrOBRy7vLmBq4tAn'}) 
downloaded.GetContentFile('train.csv')

### Using Glove Embedding to reduce the number of features for  predictions. 

Reference link: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

About Different Embeddings: https://www.kaggle.com/sbongo/do-pretrained-embeddings-give-you-the-extra-edge

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [0]:
all_df = pd.read_csv("train.csv")

In [0]:
all_df.shape

(1306122, 3)

In [0]:
train_df, test_df= train_test_split(all_df, test_size=0.2)

In [0]:
train_df, val_df= train_test_split(train_df, test_size=0.1)

In [0]:
train_df.head()

,qid,question_text,target
593525,743f651fd75e4fcce9fb,If Trump business starts deteriorating because...,0
325754,3fd8cfdd08f9d276a2a5,How many mobile app games require a push of a ...,0
573123,704de5a9be36294d96ee,Could you suggest me some trending/funny/most ...,0
1232078,f171970eae8063042ffb,Is their any success story who failed in study...,0
611008,77a6ff35aede0e3f13d3,What is alternative code for URL Shortener in ...,0


In [0]:
train_df.shape

(940407, 3)

In [0]:
train_df[train_df['target']==1].head()

,qid,question_text,target
1289250,fcada72b79b8b2f4a099,Does Amit Shah and Narendra Modi are in homose...,1
1179452,e72045eceb8db906878c,"Since I have an IQ of 369, what should my Elo ...",1
664250,8216a57c76b415a77aed,Why do Tamil Iyers perceive Iyengars to be luc...,1
290017,38cdfd02ed8e733570b6,Why is ancient slavery considered morally wron...,1
1057347,cf2e9b31a1ae5be3a7b3,Do Muslim girls love Hindu cock?,1


In [0]:
# embdedding setup
downloaded = drive.CreateFile({'id':'1fVwAHGvIslyZwKNsvg0ULHTp5R6iTZQD'}) 
downloaded.GetContentFile('glove.840B.300d.txt')
embeddings_index = {}
f = open('glove.840B.300d.txt')
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

2196017it [02:56, 12440.83it/s]

Found 2196016 word vectors.


In [0]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm(val_df["question_text"][:3000])])
val_y = np.array(val_df["target"][:3000])

100%|██████████| 3000/3000 [00:00<00:00, 18765.62it/s]


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate, CuDNNLSTM
from keras.optimizers import Adam
from keras.layers import Embedding

In [0]:
# Data providers
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            text_arr = np.array([text_to_array(text) for text in texts])
            yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

In [0]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(30, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
mg = batch_gen(train_df)

In [0]:
model.fit_generator(mg, epochs=20,
                    steps_per_epoch=1000,
                    validation_data=(val_vects, val_y),
                    verbose=True)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1000/1000 [==============================] - 54s 54ms/step - loss: 0.1336 - acc: 0.9491 - val_loss: 0.1284 - val_acc: 0.9440
Epoch 2/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1193 - acc: 0.9535 - val_loss: 0.1237 - val_acc: 0.9487
Epoch 3/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1144 - acc: 0.9547 - val_loss: 0.1237 - val_acc: 0.9527
Epoch 4/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1144 - acc: 0.9554 - val_loss: 0.1247 - val_acc: 0.9487
Epoch 5/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1110 - acc: 0.9565 - val_loss: 0.1175 - val_acc: 0.9513
Epoch 6/20
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1108 - acc: 0.9568 - val_loss: 0.1200 - val_acc: 0.9520
Epoch 7/20
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1069 - acc: 0.9583 - val_loss: 0.1187 - val_acc: 0.9493


In [0]:
# prediction part
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr


all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

1021it [01:50,  9.27it/s]


In [0]:
from sklearn.metrics import f1_score
y_te = (np.array(all_preds) > 0.5).astype(np.int)

In [0]:
y_te.shape

(261225,)

In [0]:
test_y.shape

(3000,)

In [0]:
test_df.columns

Index(['qid', 'question_text', 'target'], dtype='object')

In [0]:
y_test = test_df['target']

In [0]:
y_test.shape

(261225,)

In [0]:
f1_score(y_test,y_te)

0.6444728692608052

In [0]:
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score

In [0]:
accuracy_score(y_test,y_te)

0.9595674227198775

In [0]:
roc_auc_score(y_test,y_te)

0.7926266247194336

In [0]:
cohen_kappa_score(y_test,y_te)

0.6231476017131232

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 30, 128)           187392    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               99328     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 129       
Total params: 286,849
Trainable params: 286,849
Non-trainable params: 0
_________________________________________________________________
